# Add folders containing data and algorithms to current working path.

In [1]:
%%time
import sys, os
# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
# add Algorithm folder to current working path in order to access the functions inside the folder ´Algorithms´
sys.path.append(parent_directory+"/General_Functions")

CPU times: user 199 µs, sys: 154 µs, total: 353 µs
Wall time: 266 µs


# Double Gyre 

## Import Double Gyre Data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../Data/double_gyre.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

CPU times: user 156 ms, sys: 40.3 ms, total: 197 ms
Wall time: 200 ms


## Data/Parameters for Dynamical System

In [3]:
%%time
import numpy as np

# Number of cores to be used for parallel computing
Ncores = 8

# Incompressible/Compressible flow. {True, False}
Incompressible = True

# Periodic boundary conditions
periodic_x = False
periodic_y = False
Periodic = [periodic_x, periodic_y]

## Compute Meshgrid
X, Y = np.meshgrid(x, y)

# List of parameters of the flow.
params_data = {"X": X, "Y": Y, "Time": time, "U": U, "V": V, "Ncores": Ncores, 
               "Incompressible": Incompressible, "Periodic": Periodic}

(21, 41)
CPU times: user 232 µs, sys: 73 µs, total: 305 µs
Wall time: 277 µs


## Spatio-Temporal Domain of Dynamical System

In [4]:
%%time
# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 10

# time step-size
dt = .1

time = np.arange(t0, tN+dt, dt)

# longitudinal and latitudinal boundaries (in degrees)
xmin = 0
xmax = 2
ymin = 0
ymax = 1

# spacing of meshgrid (in degrees)
dx = 0.01
dy = 0.01

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

params_DS = {"time": time, "X_domain": X_domain, "Y_domain": Y_domain}

CPU times: user 391 µs, sys: 272 µs, total: 663 µs
Wall time: 423 µs


In [5]:
%%time
# Initialize Dynamical System
from ipynb.fs.defs.DynamicalSystem import *
DS = Dynamical_System(params_data, params_DS)

CPU times: user 84.1 ms, sys: 18.7 ms, total: 103 ms
Wall time: 109 ms


## Interpolate Velocity

In [6]:
%%time
# Interpolate velocity data using cubic spatial interpolation
DS._Interpolation_velocity("cubic")

CPU times: user 45.7 ms, sys: 21.7 ms, total: 67.4 ms
Wall time: 71.8 ms


## Gradient of Flow map over meshgrid of Initial Conditions

In [ ]:
%%time
# aux_grid = True --> Use auxiliary grid for numerical computation of gradient.
# Otherwise aux_grid = False.
aux_grid = True

grad_Fmap_grid = DS._grad_Fmap_grid(aux_grid)

  0%|          | 0/101 [00:00<?, ?it/s]

## Compute Cauchy-Green strain tensor over meshgrid of Initial Conditions

In [ ]:
B, C = DS._cauchy_green_strain(grad_Fmap_grid)

## Compute Finite Time Lyapunov Exponent

In [ ]:
from ipynb.fs.defs.FTLE import _FTLE
FTLE_min, FTLE_max = _FTLE(C, tN-t0, Incompressible)

In [ ]:
######################## PLOT RESULTS ########################

import matplotlib.pyplot as plt

# Figure/Axis
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()

# Contourplot of TSE over meshgrid of initial conditions
cax = ax.contourf(X_domain, Y_domain, FTLE_max, cmap = "gist_rainbow_r", levels = 600)

# Axis Labels
ax.set_xlabel("x", fontsize = 18)
ax.set_ylabel("y", fontsize = 18)

# Ticks
ax.set_xticks(np.arange(np.min(X_domain), np.max(X_domain), .2))
ax.set_yticks(np.arange(np.min(Y_domain), np.max(Y_domain), .1))

# Colorbar
cbar = fig.colorbar(cax, ticks = np.linspace(0, 1, 11))
cbar.ax.set_ylabel(r'$(\dfrac{1}{d})$', rotation = 0, labelpad = 10)

# Title
ax.set_title(r'$ FTLE $'+f'$_{{{int(time[0])}d}}^{{{int(time[-1])}d}}$', fontsize = 18)

plt.show()